In [ ]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster
import os

csv_folder = "scraper/agences_bnp"
all_data = []

# 📥 Lecture de tous les CSV + nettoyage
for file in os.listdir(csv_folder):
    if file.endswith(".csv"):
        path = os.path.join(csv_folder, file)
        try:
            df = pd.read_csv(path)
            df.columns = [col.strip().lower() for col in df.columns]
            if {"nom", "adresse", "code_postal", "latitude", "longitude"}.issubset(df.columns):
                df.dropna(subset=["latitude", "longitude"], inplace=True)
                all_data.append(df)
                print(f"✅ {file} : {len(df)} lignes")
        except Exception as e:
            print(f"❌ {file} : {e}")

if not all_data:
    print("❗ Aucun CSV valide")
    exit()

df_all = pd.concat(all_data, ignore_index=True)
print(f"📦 Total : {len(df_all)} agences")

# 🗺️ Carte de base (centrée sur la France)
m = folium.Map(location=[46.5, 2.5], zoom_start=6, control_scale=True)

# ⚡ Cluster rapide, homogène, style Google Maps
FastMarkerCluster(
    data=df_all[["latitude", "longitude"]].values.tolist()
).add_to(m)

# 💾 Export HTML
m.save("carte_agences_bnp_google_style.html")
print("🗺️ Carte exportée : carte_agences_bnp_google_style.html")


✅ Carte exportée : carte_agences_bnp_verte.html
